In [1]:
!pip install torch joblib


In [3]:
from google.colab import files
uploaded = files.upload()
list (uploaded.keys())

Saving recsys_model_updated.pth to recsys_model_updated.pth
Saving lbl_song_updated.pkl to lbl_song_updated.pkl
Saving lbl_user_updated.pkl to lbl_user_updated.pkl


['recsys_model_updated.pth', 'lbl_song_updated.pkl', 'lbl_user_updated.pkl']

In [4]:
import joblib

lbl_user = joblib.load("lbl_user_updated.pkl")
lbl_song = joblib.load("lbl_song_updated.pkl")

id2song = dict(enumerate(lbl_song.classes_))
print("Users:", len(lbl_user.classes_), "| Songs:", len(lbl_song.classes_))


Users: 200000 | Songs: 127771


In [5]:
import torch
import torch.nn as nn

class RecSysModel(nn.Module):
    def __init__(self, n_users, n_songs):
        super().__init__()
        self.user_embeds = nn.Embedding(n_users, 32)
        self.song_embeds = nn.Embedding(n_songs, 32)
        self.out = nn.Linear(64, 1)

    def forward(self, users, songs):
        u = self.user_embeds(users)
        s = self.song_embeds(songs)
        x = torch.cat([u, s], dim=1)
        return self.out(x)


In [8]:
n_users = len(lbl_user.classes_)
n_songs = len(lbl_song.classes_)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RecSysModel(n_users, n_songs).to(device)
model.load_state_dict(torch.load("recsys_model_updated.pth", map_location=device))
model.eval()

RecSysModel(
  (user_embeds): Embedding(200000, 32)
  (song_embeds): Embedding(127771, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
def recommend_songs_for_user(user_name, model, top_k=5):
    # Handle both string and integer user IDs
    try:
        user_name_int = int(user_name)
    except ValueError:
        user_name_int = None # Not a valid integer ID

    if user_name not in lbl_user.classes_ and (user_name_int is None or user_name_int not in lbl_user.classes_):
        return ["User not found."]

    # Use the original user_name string for transformation if it exists, otherwise use the integer
    user_id = lbl_user.transform([user_name if user_name in lbl_user.classes_ else user_name_int])[0]
    all_song_ids = torch.arange(len(id2song))
    user_tensor = torch.tensor([user_id] * len(all_song_ids))

    # Move tensors to the same device as the model
    device = next(model.parameters()).device
    user_tensor = user_tensor.to(device)
    all_song_ids = all_song_ids.to(device)

    with torch.no_grad():
        preds = model(user_tensor, all_song_ids).squeeze()

    top_ids = torch.topk(preds, top_k).indices.tolist()
    return [id2song[i] for i in top_ids]

In [16]:
user = input("Enter a user ID: ")
recs = recommend_songs_for_user(user, model, top_k=5)

print(f"\nTop 5 recommended corresponding song_ids for {user}:")
for r in recs:
    print("-", r)

Enter a user ID: 5

Top 5 recommended corresponding song_ids for 5:
- 67423
- 49084
- 19169
- 96148
- 124558
